# Item 22: Reduce Visual Noise with Variable Positional Arguments

In [1]:
# Accepting a variable number of positional arguments can make a function call clearer and reduce visual noise
# Say we want to log some debugging info. With a fixed number of arguments, we would need a functio that takes a 
# message and a list of values
def log(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', [1, 2])
log('Hi there', [])

My numbers are: 1, 2
Hi there


Having to pass an empty list when we have no values to log is cumbersome an noisy. It'd be better to leave out the second argment entirely. We can do this in Python by prefixing the last positional parameter name with `*`.

In [2]:
# The first parameter for the log message is required, whereas any number of subsequent positional arguments are
# optional. The function body doesn't need to change, only the callers do
def log(message, *values): #The only difference
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', 1, 2)
log('Hi there') # Much better

My numbers are: 1, 2
Hi there


In [5]:
# If we already have a sequence (like a list) and want to call a variadic function like log, we can do this by
# using the * operator. This instructs Python to pass items from the sequence as positional a to the function
favorites = [7, 33, 99]
log('Favorite colors', *favorites)

Favorite colors: 7, 33, 99


There are two problems with accepting a variable number of positional arguments:

The first issue is that these positional arguments are always turned into a tuple before they are passed to a function. This means that if a caller of the function uses the `*` operator on a generator, it will be iterated until its exhausted. The resulting tuple includes every value from the generator, which could consume a lot of memory and cause the program to crash

In [6]:
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)

it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


Functions that accept `*args` is best for situations where you know the number of inputs in the argument list will be reasonably small. `*args` is ideal for function calls that pass many literals or variable names together. It's primarily for the convinience of the programmer and the readablity of the code.

The second issue with `*args` is that you can't add new positional arguments to a function in the future without migrating every caller. If we try to add a new positional argument in the front of the argument list, existing callers will subtly break if they aren't updated.

In [7]:
def log(sequence, message, *values):
    if not values:
        print(f'{sequence} - {message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{sequence} - {message}: {values_str}')

log(1, 'Favorites', 7, 33)     # New with *args OK
log(1, 'Hi there')             # New message only
log('Favorite numbers', 7, 33) # Old usage breaks

1 - Favorites: 7, 33
1 - Hi there
Favorite numbers - 7: 33


The problem with code above is that third call to `log` used `7` as the `message` parameter because a `sequence` argument was not given. To avoid this possibility entirely, we should use keyword-only arguments when we want to extend functions that accept `*args`. To be more defensive, use type annotations as well.